# Task 2. Extending the prefix tree functionality


Implement two additional methods for the Trie class:
- `count_words_with_suffix(pattern)` to count the number of words ending with a given pattern;
- `has_prefix(prefix)` to check for the presence of words with a given prefix.

In [1]:
from task_2_examples.trie import Trie
import time

In [2]:
class Homework(Trie):
    """Trie with additional methods for searching suffixes and prefixes."""

    def count_words_with_suffix(self, pattern: str) -> int:
        """
        Returns the number of keys that end with the given suffix.
        Args:
            pattern (str): The suffix to search for.
        Returns:
            int: The number of keys ending with the given suffix.
        Raises:
            TypeError: If the pattern is not a string.
        """
        if not isinstance(pattern, str):
            raise TypeError("pattern must be a string")

        count = 0
        stack = [(self.root, "")]

        while stack:
            node, prefix = stack.pop()
            if node.value is not None and prefix.endswith(pattern):
                count += 1
            for char, child in node.children.items():
                stack.append((child, prefix + char))

        return count

    def has_prefix(self, prefix: str) -> bool:
        """
        Returns True if there is at least one word with the given prefix.
        Args:
            prefix (str): The prefix to search for.
        Returns:
            bool: True if there is at least one word with the given prefix, False otherwise.
        Raises:
            TypeError: If the prefix is not a string.
        """
        if not isinstance(prefix, str):
            raise TypeError("prefix must be a string")

        if prefix == "":
            return not self.is_empty()

        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]

        if node.value is not None:
            return True

        stack = list(node.children.values())
        while stack:
            child = stack.pop()
            if child.value is not None:
                return True
            stack.extend(child.children.values())

        return False

### Testing the methods

In [3]:
homework = Homework()
dataset = [
    "apple",
    "application",
    "banana",
    "band",
    "cat",
    "catalog",
]
for index, word in enumerate(dataset):
    homework.put(word, index)

# Checking suffixes
assert homework.count_words_with_suffix("le") == 1  # apple
assert homework.count_words_with_suffix("e") == 1  # apple
assert homework.count_words_with_suffix("ion") == 1  # application
assert homework.count_words_with_suffix("a") == 1  # banana
assert homework.count_words_with_suffix("og") == 1  # catalog
assert homework.count_words_with_suffix("") == len(dataset)  # empty suffix matches all words

# Checking prefixes
assert homework.has_prefix("app") is True  # apple, application
assert homework.has_prefix("ban") is True  # banana, band
assert homework.has_prefix("bat") is False
assert homework.has_prefix("cat") is True  # cat, catalog
assert homework.has_prefix("") is True  # empty prefix is acceptable for non-empty trie

# Checking error handling
try:
    homework.count_words_with_suffix(123)
except TypeError as error:
    print(f"suffix error: {error}")

try:
    homework.has_prefix(None)
except TypeError as error:
    print(f"prefix error: {error}")

suffix error: pattern must be a string
prefix error: prefix must be a string


### Additional analysis

In [4]:
print("Checking count_words_with_suffix method:")
print("-" * 60)

print(f"\nWords in the trie: {homework.keys()}")

test_suffixes = ["e", "ion", "a", "og", "le", "d", ""]
for suffix in test_suffixes:
    count = homework.count_words_with_suffix(suffix)
    matching_words = [word for word in homework.keys() if word.endswith(suffix)]
    print(f"Suffix '{suffix}': {count} words - {matching_words}")


print("\nChecking has_prefix method:")
print("-" * 60)

test_prefixes = ["app", "ban", "bat", "cat", "c", "a", ""]
for prefix in test_prefixes:
    has_prefix = homework.has_prefix(prefix)
    matching_words = [word for word in homework.keys() if word.startswith(prefix)]
    print(f"Prefix '{prefix}': {has_prefix} - {matching_words}")

Checking count_words_with_suffix method:
------------------------------------------------------------

Words in the trie: ['apple', 'application', 'banana', 'band', 'cat', 'catalog']
Suffix 'e': 1 words - ['apple']
Suffix 'ion': 1 words - ['application']
Suffix 'a': 1 words - ['banana']
Suffix 'og': 1 words - ['catalog']
Suffix 'le': 1 words - ['apple']
Suffix 'd': 1 words - ['band']
Suffix '': 6 words - ['apple', 'application', 'banana', 'band', 'cat', 'catalog']

Checking has_prefix method:
------------------------------------------------------------
Prefix 'app': True - ['apple', 'application']
Prefix 'ban': True - ['banana', 'band']
Prefix 'bat': False - []
Prefix 'cat': True - ['cat', 'catalog']
Prefix 'c': True - ['cat', 'catalog']
Prefix 'a': True - ['apple', 'application']
Prefix '': True - ['apple', 'application', 'banana', 'band', 'cat', 'catalog']


### Testing on big data sets

In [10]:
large_trie = Homework()
words = [
    "test", "testing", "tester", "tested", "testament",
    "programming", "program", "programmer", "programmatic",
    "data", "database", "datatype", "dataset",
    "algorithm", "algorithmic", "algorithms",
    "structure", "structural", "structured",
    "performance", "performant", "perform",
    "efficiency", "efficient", "efficiently"
]

# Adding each word 100000 times with different indices
for i in range(1000000):
    for j, word in enumerate(words):
        large_trie.put(f"{word}{i}", i * len(words) + j)

print(f"Trie size: {large_trie.size} words")

# Speed test for count_words_with_suffix
print("\nSpeed test for count_words_with_suffix:")
print("-" * 60)

test_cases = ["ing", "er", "t", "0", "99"]
for suffix in test_cases:
    start_time = time.time()
    count = large_trie.count_words_with_suffix(suffix)
    elapsed = time.time() - start_time
    print(f"Suffix '{suffix}': {count} words, time: {elapsed*1000:.2f} ms")

# Speed test for# Speed test for has_prefix
print("\nSpeed test for has_prefix:")
print("-" * 60)

test_prefixes = ["test", "prog", "data", "algo", "xyz"]
for prefix in test_prefixes:
    start_time = time.time()
    result = large_trie.has_prefix(prefix)
    elapsed = time.time() - start_time
    print(f"Prefix '{prefix}': {result}, time: {elapsed*1000:.2f} ms")

Trie size: 25000000 words

Speed test for count_words_with_suffix:
------------------------------------------------------------
Suffix 'ing': 0 words, time: 6453.74 ms
Suffix 'er': 0 words, time: 6474.78 ms
Suffix 't': 0 words, time: 6447.82 ms
Suffix '0': 2500000 words, time: 6619.25 ms
Suffix '99': 250000 words, time: 6549.11 ms

Speed test for has_prefix:
------------------------------------------------------------
Prefix 'test': True, time: 0.00 ms
Prefix 'prog': True, time: 0.00 ms
Prefix 'data': True, time: 0.00 ms
Prefix 'algo': True, time: 0.00 ms
Prefix 'xyz': False, time: 0.00 ms


### Edge case testing

In [6]:
print("Edge cases:")
print("-" * 60)

# Empty trie
empty_trie = Homework()
print(f"\n1. Empty trie:")
print(f"   count_words_with_suffix(''): {empty_trie.count_words_with_suffix('')}")
print(f"   has_prefix(''): {empty_trie.has_prefix('')}")
print(f"   has_prefix('a'): {empty_trie.has_prefix('a')}")

# Single-character words
single_trie = Homework()
single_trie.put("a", 1)
single_trie.put("b", 2)
single_trie.put("c", 3)
print(f"\n2. Single-character words (a, b, c):")
print(f"   count_words_with_suffix('a'): {single_trie.count_words_with_suffix('a')}")
print(f"   count_words_with_suffix('b'): {single_trie.count_words_with_suffix('b')}")
print(f"   has_prefix('a'): {single_trie.has_prefix('a')}")
print(f"   has_prefix('ab'): {single_trie.has_prefix('ab')}")

# Nested words
nested_trie = Homework()
nested_trie.put("a", 1)
nested_trie.put("ab", 2)
nested_trie.put("abc", 3)
nested_trie.put("abcd", 4)
print(f"\n3. Nested words (a, ab, abc, abcd):")
print(f"   count_words_with_suffix('a'): {nested_trie.count_words_with_suffix('a')}")
print(f"   count_words_with_suffix('d'): {nested_trie.count_words_with_suffix('d')}")
print(f"   has_prefix('a'): {nested_trie.has_prefix('a')}")
print(f"   has_prefix('abc'): {nested_trie.has_prefix('abc')}")
print(f"   has_prefix('abcde'): {nested_trie.has_prefix('abcde')}")

# Case sensitivity
case_trie = Homework()
case_trie.put("Apple", 1)
case_trie.put("apple", 2)
print(f"\n4. Case sensitivity (Apple, apple):")
print(f"   count_words_with_suffix('e'): {case_trie.count_words_with_suffix('e')}")
print(f"   count_words_with_suffix('E'): {case_trie.count_words_with_suffix('E')}")
print(f"   has_prefix('App'): {case_trie.has_prefix('App')}")
print(f"   has_prefix('app'): {case_trie.has_prefix('app')}")

# Error handling test
print(f"\n5. Error handling:")
error_trie = Homework()
error_trie.put("test", 1)

errors_caught = []
try:
    error_trie.count_words_with_suffix(None)
except TypeError:
    errors_caught.append("None for suffix")

try:
    error_trie.count_words_with_suffix(123)
except TypeError:
    errors_caught.append("int for suffix")

try:
    error_trie.has_prefix([])
except TypeError:
    errors_caught.append("list for prefix")

try:
    error_trie.has_prefix({"key": "value"})
except TypeError:
    errors_caught.append("dict for prefix")

print(f"   caught: {', '.join(errors_caught)}")

Edge cases:
------------------------------------------------------------

1. Empty trie:
   count_words_with_suffix(''): 0
   has_prefix(''): False
   has_prefix('a'): False

2. Single-character words (a, b, c):
   count_words_with_suffix('a'): 1
   count_words_with_suffix('b'): 1
   has_prefix('a'): True
   has_prefix('ab'): False

3. Nested words (a, ab, abc, abcd):
   count_words_with_suffix('a'): 1
   count_words_with_suffix('d'): 1
   has_prefix('a'): True
   has_prefix('abc'): True
   has_prefix('abcde'): False

4. Case sensitivity (Apple, apple):
   count_words_with_suffix('e'): 2
   count_words_with_suffix('E'): 0
   has_prefix('App'): True
   has_prefix('app'): True

5. Error handling:
   caught: None for suffix, int for suffix, list for prefix, dict for prefix


## Algorithmic complexity

**`count_words_with_suffix(pattern)`:**
- Time complexity: $O(n)$, where $n$ is the number of nodes in the tree.
- Space complexity: $O(h)$, where $h$ is the height of the tree (for a stack).
- The implementation uses iterative DFS to traverse the entire tree.
- Results on 25,000,000 words: **~6500 ms** (6.5 seconds)


**`has_prefix(prefix)`:**
- Time complexity: $O(m + k)$, where $m$ is the length of the prefix, $k$ is the number of nodes in the subtree.
- Space complexity: $O(k)$ for a stack in the worst case.
- Optimization: the search stops after finding the first word.
- Results on 25,000,000 words: **0 ms** (instant)